In [1]:
import sage.misc.banner; sage.misc.banner.banner()

┌────────────────────────────────────────────────────────────────────┐
│ SageMath version 10.1, Release Date: 2023-08-20                    │
│ Using Python 3.11.1. Type "help()" for help.                       │
└────────────────────────────────────────────────────────────────────┘


In [3]:
p = 15*2^27+1; print(p)
F = GF(p)
R.<x> = F[]
FE.<u> = F.extension(x^4 - 11)
Q.<v> = FE[]

2013265921


In [4]:
two_adicity = dict(factor(p-1))[2]; two_adicity

27

In [5]:
g = F.multiplicative_generator(); print(g)
g2 = g^((p-1)/2^two_adicity); print(g2)
factor(g2.multiplicative_order())

31
440564289


2^27

In [6]:
def two_adic_gen(size_bits):
    return g2^(two_adicity - size_bits)
def two_adic_subgroup(size_bits):
    return two_adic_gen(size_bits).powers(2^size_bits)

In [7]:
def dft(poly, added_bits=0, shift=1):
    len_bits = log(poly.degree() + 1, 2) + added_bits
    dom = [shift * (g2^(two_adicity - len_bits))^i for i in range(2^len_bits)]
    return [poly(x) for x in dom]

In [8]:
def idft(evals, shift=1):
    len_bits = log(len(evals), 2)
    dom = [shift * (g2^(two_adicity - len_bits))^i for i in range(2^len_bits)]
    return evals[0].parent().polynomial_ring().lagrange_polynomial(zip(dom, evals))

In [9]:
LEN_BITS = 4
LEN = 1 << LEN_BITS
DEG = LEN - 1

In [126]:
d = FE.degree()
D = 15
poly = R.random_element(D); poly

965207358*x^15 + 1786849254*x^14 + 1791697776*x^13 + 464349102*x^12 + 1808633257*x^11 + 712419247*x^10 + 1665100900*x^9 + 1842107603*x^8 + 854659407*x^7 + 669364161*x^6 + 1128439517*x^5 + 1231558625*x^4 + 1655300014*x^3 + 176232450*x^2 + 1231326767*x + 952380526

In [127]:
alpha = FE.random_element()

In [128]:
def old_quotient_poly(poly, alpha, poly_at_alpha):
    return (poly - poly_at_alpha) / (x - alpha)

good_qp = old_quotient_poly(poly, alpha, poly(alpha))
assert good_qp.is_integral() and good_qp.numerator().degree() == D - 1

bad_qp = old_quotient_poly(poly, alpha, poly(alpha) + 1)
assert not bad_qp.is_integral()

In [129]:
# Liam's trick

# prover and verifier compute m(X)
m = minpoly(alpha)
assert m.degree() == d and m.is_monic()

# prover provides r(X)
r = poly % m
assert r.degree() == d - 1

# ldt
qp = (poly - r) / m
assert qp.is_integral() and qp.numerator().degree() == D - d

# ood check
assert r(alpha) == poly(alpha)

In [130]:
assert r(alpha.frobenius()) == poly(alpha.frobenius())

In [131]:
r

357547801*x^3 + 925973360*x^2 + 452312119*x + 1614737475

In [132]:
# O(D ~d^2)
p_a = poly(alpha)
# O(d^2)
r = Q.lagrange_polynomial([(alpha.frobenius(i), p_a.frobenius(i)) for i in range(d)])
assert r.change_ring(F).change_variable_name(x) == (poly % m)

In [133]:
alpha

1264455760*u^3 + 827593842*u^2 + 277371702*u + 1455741459

In [134]:
p_a

1151226591*u^3 + 976625391*u^2 + 913580271*u + 43457219

In [135]:
def frobs(x):
    return [x.frobenius(i) for i in range(x.parent().degree())]
frobs(alpha)

[1264455760*u^3 + 827593842*u^2 + 277371702*u + 1455741459,
 36126823*u^3 + 1185672079*u^2 + 1849141035*u + 1455741459,
 748810161*u^3 + 827593842*u^2 + 1735894219*u + 1455741459,
 1977139098*u^3 + 1185672079*u^2 + 164124886*u + 1455741459]

In [137]:
'''
L(x) = sum(y_i l_i(x))
l_i(x) = l(x) * w_i / (x - x_i)
l(x) = prod(x - x_i)
so
L(x) = l(x) sum(y_i * w_i / (x - x_i))
'''
def l(xs, i):
    x = polygen(xs[0])
    return prod(
        (x - xs[j]) / (xs[i] - xs[j])
        for j in range(len(xs))
        if i != j
    )

G = FE.galois_group()

xs = frobs(alpha)
ys = frobs(p_a)
for i in range(len(xs)):
    l_i = l(xs, i)
    
    for j in range(len(xs)):
        if i == j:
            assert l_i(xs[j]) == 1
        else:
            assert l_i(xs[j]) == 0
    
    if i == 0:
        for k in range(len(xs)):
            print(k, Q([G[k](coeff) for coeff in (ys[i] * l_i).coefficients()]))
        print()
    
    print(i, ys[i] * l_i)
    #print(G[0](ys[i] * l_i))

0 (616141894*u^3 + 1858831400*u^2 + 1578015625*u + 1599336391)*v^3 + (688248041*u^3 + 1070808247*u^2 + 1927894725*u + 231493340)*v^2 + (2005622016*u^3 + 615181997*u^2 + 862601011*u + 616394510)*v + 665125299*u^3 + 1939317420*u^2 + 151422267*u + 907000849
1 (183766845*u^3 + 154434521*u^2 + 800365690*u + 1599336391)*v^3 + (1768339845*u^3 + 942457674*u^2 + 1299511171*u + 231493340)*v^2 + (255737073*u^3 + 1398083924*u^2 + 474191180*u + 616394510)*v + 866677836*u^3 + 73948501*u^2 + 1092601613*u + 907000849
2 (1397124027*u^3 + 1858831400*u^2 + 435250296*u + 1599336391)*v^3 + (1325017880*u^3 + 1070808247*u^2 + 85371196*u + 231493340)*v^2 + (7643905*u^3 + 615181997*u^2 + 1150664910*u + 616394510)*v + 1348140622*u^3 + 1939317420*u^2 + 1861843654*u + 907000849
3 (1829499076*u^3 + 154434521*u^2 + 1212900231*u + 1599336391)*v^3 + (244926076*u^3 + 942457674*u^2 + 713754750*u + 231493340)*v^2 + (1757528848*u^3 + 1398083924*u^2 + 1539074741*u + 616394510)*v + 1146588085*u^3 + 73948501*u^2 + 920664308

In [138]:
def interp_naive(xs, ys):
    return sum(ys[i] * l(xs, i) for i in range(len(xs)))

interp_naive(frobs(alpha), frobs(p_a))

357547801*x^3 + 925973360*x^2 + 452312119*x + 1614737475

In [136]:
def interp_ref(xs, ys):
    r = Q.lagrange_polynomial(zip(xs, ys))
    return [F(c) for c in r.coefficients()]

interp_ref(frobs(alpha), frobs(p_a))

[1614737475, 452312119, 925973360, 357547801]

In [167]:
def binomial_expand(xs):
    coeffs = [1] + [0] * len(xs)
    for i, x in enumerate(xs):
        for j in range(i+1, 0, -1):
            coeffs[j] = coeffs[j-1] - x * coeffs[j]
        coeffs[0] *= -x
    return coeffs

def precompute(alpha):
    xs = frobs(alpha)
    w_alpha = prod(xs[0] - xs[i] for i in range(1, len(xs)))
    return [c / w_alpha for c in binomial_expand(xs[1:])]

def interp_smart(alpha, p_alpha):
    n = alpha.parent().degree()
    l_alpha = precompute(alpha)
    
    L_alpha = [p_alpha * c for c in l_alpha]
    poly = L_alpha
    
    for i in range(1, n):
        L_alpha = [c.frobenius() for c in L_alpha]
        poly = [a + b for (a, b) in zip(poly, L_alpha)]
    
    return poly

pp = interp_smart(alpha, p_a)
assert pp == interp_ref(frobs(alpha), frobs(p_a))

In [155]:
list(range(1,0,-1))

[1]

In [101]:
#ws = [1/prod(xs[i] - xs[j] for j in range(n) if i != j) for i in range(n)]
#print(len(ws))
#print(ws[0].parent())

In [114]:
L_alpha

NameError: name 'L_alpha' is not defined

In [80]:
r = poly % m; r

1659244056*x^3 + 166059883*x^2 + 690758274*x + 1383548857

In [81]:
assert r(alpha) == poly(alpha)

In [90]:
ys = [r(x) for x in two_adic_subgroup(4)]; print(ys)

[1886345149, 1884148850, 742119584, 1791904383, 531044336, 79323216, 775959677, 936277647, 420863385, 1125218146, 1333955626, 702806318, 525901186, 1438232529, 106639216, 875576678]


In [59]:
[x for x in range(3)]

[0, 1, 2]

In [85]:
two_adic_subgroup(4)

[1,
 1474969731,
 1498750007,
 1466872449,
 723613219,
 1896673953,
 1906280669,
 10355940,
 1175043036,
 1572493914,
 747363527,
 1225193342,
 572243252,
 1611250077,
 830937851,
 688293274]

In [89]:
z = FE.random_element()
[z.frobenius(i) for i in range(10)]

[1861356301*u^3 + 1147371388*u^2 + 52573771*u + 480637679,
 1808942986*u^3 + 865894533*u^2 + 1823620891*u + 480637679,
 151909620*u^3 + 1147371388*u^2 + 1960692150*u + 480637679,
 204322935*u^3 + 865894533*u^2 + 189645030*u + 480637679,
 1861356301*u^3 + 1147371388*u^2 + 52573771*u + 480637679,
 1808942986*u^3 + 865894533*u^2 + 1823620891*u + 480637679,
 151909620*u^3 + 1147371388*u^2 + 1960692150*u + 480637679,
 204322935*u^3 + 865894533*u^2 + 189645030*u + 480637679,
 1861356301*u^3 + 1147371388*u^2 + 52573771*u + 480637679,
 1808942986*u^3 + 865894533*u^2 + 1823620891*u + 480637679]

In [95]:
G[0]

1

In [116]:
zz = Q([1, 2, 3]); zz

3*v^2 + 2*v + 1

In [141]:
zz.map_coefficients

<built-in method map_coefficients of sage.rings.polynomial.polynomial_zz_pex.Polynomial_ZZ_pEX object at 0x1425ff340>

In [142]:
lst = [1,2,3]
lst2 = lst + [4]
lst2[0] = 5

In [143]:
lst2

[5, 2, 3, 4]

In [144]:
lst

[1, 2, 3]

In [ ]:
zip